In [0]:
# Databricks Notebook: Silver -> Gold (FNG unified one-table)
from pyspark.sql import functions as F, Window

spark.sql("SET spark.sql.session.timeZone=UTC")

CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
SILVER  = f"{CATALOG}.{SCHEMA}.silver_fear_greed"
GOLD    = f"{CATALOG}.{SCHEMA}.gold_fear_greed"   # 단일 Gold

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {GOLD} (
  ts_utc       TIMESTAMP,   -- event_time
  dt           DATE,
  value        INT,
  value_class  STRING,
  ma7          DOUBLE,
  ma30         DOUBLE,
  z30          DOUBLE,
  d1_change    INT,
  d7_change    INT,
  streak_days  INT          -- 해당 날짜 기준 연속일수
) USING DELTA
PARTITIONED BY (dt)
""")

s = (spark.table(SILVER)
        .select(
            F.col("event_time").alias("ts_utc"),
            F.col("dt"),
            F.col("index_value").alias("value"),
            F.col("value_classification").alias("value_class")
        ))

# 롤링 윈도우(시간 오름차순)
w_asc  = Window.orderBy("ts_utc").rowsBetween(Window.unboundedPreceding, Window.currentRow)
w7     = Window.orderBy("ts_utc").rowsBetween(-6, 0)
w30    = Window.orderBy("ts_utc").rowsBetween(-29, 0)

g = (s
    # 이동평균 / z-score
    .withColumn("ma7",  F.avg("value").over(w7))
    .withColumn("ma30", F.avg("value").over(w30))
    .withColumn("std30",F.stddev_samp("value").over(w30))
    .withColumn("z30",  F.when(F.col("std30").isNull() | (F.col("std30")==0), F.lit(None).cast("double"))
                         .otherwise((F.col("value")-F.col("ma30"))/F.col("std30")))
    .drop("std30")
    # 변화량(전일/7일 전)
    .withColumn("d1_change", (F.col("value") - F.lag("value", 1).over(Window.orderBy("ts_utc"))).cast("int"))
    .withColumn("d7_change", (F.col("value") - F.lag("value", 7).over(Window.orderBy("ts_utc"))).cast("int"))
    # streak 계산: 동일 등급 지속일수
    .withColumn("lag_class", F.lag("value_class", 1).over(Window.orderBy("ts_utc")))
    .withColumn("chg_flag",  F.when(F.col("value_class")==F.col("lag_class"), F.lit(0)).otherwise(F.lit(1)))
    .withColumn("grp",       F.sum("chg_flag").over(w_asc))
    .withColumn("streak_days", F.row_number().over(Window.partitionBy("grp").orderBy("ts_utc")))
    .select("ts_utc","dt","value","value_class","ma7","ma30","z30","d1_change","d7_change","streak_days")
)

# 단일 Gold 갱신(전체 덮어쓰기)
g.write.mode("overwrite").option("mergeSchema","true").saveAsTable(GOLD)
print(f"[FNG GOLD] refreshed -> {GOLD}")
